In [1]:
import time
import random
import selenium
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import shutil
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
driver = selenium.webdriver.Chrome(ChromeDriverManager().install())

website = "https://www.dpreview.com/reviews/image-comparison?attr18=daylight&attr13_0=apple_iphonex&attr13_1=apple_iphonex&attr13_2=apple_iphonex&attr13_3=apple_iphonex&attr15_0=jpeg&attr15_1=jpeg&attr15_2=jpeg&attr15_3=jpeg&attr16_0=32&attr16_1=32&attr16_2=32&attr16_3=32&normalization=full&widget=1&x=0&y=0"

driver.get(website)
driver.implicitly_wait(5)

In [13]:
# login

def login(driver):
    wait = WebDriverWait(driver, 2)
    wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[3]/div/div[2]/a[1]"))).click()
    driver.implicitly_wait(3)
    wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/form/div/div/div/div[1]/input"))).send_keys("login")
    driver.implicitly_wait(3)
    wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/form/div/div/div/div[2]/input"))).send_keys("password")
    wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/form/div/div/div/div[3]/span/span/input"))).click()
    time.sleep(3)

In [2]:
def download_image(driver, format):
    if format == "RAW":
        wait = WebDriverWait(driver, 2)
        wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[1]/a[2]"))).click()   
        time.sleep(0.2)
        wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[1]/a[1]"))).click()   

        still_downloading = True

        while still_downloading == True:
            time.sleep(1)
            downloading_files = 0
            for file in os.listdir("/home/user/Downloads/"):
                if file.endswith("crdownload"):
                    downloading_files += 1
                    print("Still downloading")
            if downloading_files == 0:
                still_downloading = False

    elif format == "JPEG":
        wait = WebDriverWait(driver, 2)
        wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[3]/div[1]/a"))).click()   

        still_downloading = True

        while still_downloading == True:
            time.sleep(1)
            downloading_files = 0
            for file in os.listdir("/home/user/Downloads/"):
                if file.endswith("crdownload"):
                    downloading_files += 1
                    print("Still downloading")
            if downloading_files == 0:
                still_downloading = False


def move_download(path):
    downloaded_images = os.listdir("/home/user/Downloads/")
    for img in downloaded_images:
        try:
            shutil.move("/home/user/Downloads/"+img, path)
            print("File '%s' moved successfully" % "/home/user/Downloads/"+img)
        except OSError as error:
            print("File '%s' already exists" % "/home/user/Downloads/"+img)
            os.remove("/home/user/Downloads/"+img)


In [ ]:
# set up the number of retries and the time to wait between retries
num_retries = 100
wait_time = 41
itterator_stop = 85

driver = selenium.webdriver.Chrome(ChromeDriverManager().install())
website = "https://www.dpreview.com/reviews/image-comparison?attr18=daylight&attr13_0=apple_iphonex&attr13_1=apple_iphonex&attr13_2=apple_iphonex&attr13_3=apple_iphonex&attr15_0=jpeg&attr15_1=jpeg&attr15_2=jpeg&attr15_3=jpeg&attr16_0=32&attr16_1=32&attr16_2=32&attr16_3=32&normalization=full&widget=1&x=0&y=0"
driver.get(website)
#login(driver) #login may increase the number of https requests before 429

# loop through the number of retries
for i in range(num_retries):
    try:
        driver.get(website)
        driver.implicitly_wait(5)
        wait = WebDriverWait(driver, 2)

        select = Select(wait.until(EC.visibility_of_element_located((By.XPATH,"/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[1]/div/select"))))
        device_number = len(select.options)
        # itterate over all devices
        for device_num in range(itterator_stop, device_number):
            itterator_raw = 0
            time.sleep(1)
            select = Select( wait.until(EC.visibility_of_element_located((By.XPATH,"/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[1]/div/select"))))
            time.sleep(1)
            dir_name = select.options[device_num].text
            select.select_by_visible_text(dir_name)
            # select by visible text
            print("opt_format: ", dir_name)

            dir_name = dir_name.replace(" ", "_")
            dir_name = dir_name.replace("(", "_")
            dir_name = dir_name.replace(")", "_")

            # create directory for each device
            path_device = os.path.join("/home/user/scratch/dpreview/", dir_name)
            print("path: ", path_device)

            try:
                os.mkdir(path_device)
                print("Directory '%s' created successfully" % path_device)
            except OSError as error:
                print("Directory '%s' already exists" % path_device)

            # find element of list for jpeg or raw
            select_format = Select( wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[2]/div/select"))))
            time.sleep(1)
            format_number = len(select_format.options)
            print("format_number ",format_number)

            for format_itterator in range(itterator_raw, format_number):
                itterator_iso = 0
                time.sleep(1)
                select_format = Select( wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[2]/div/select"))))
                time.sleep(1)
                format = select_format.options[format_itterator].text
                select_format.select_by_visible_text(format)
                
                print("format:",format)
                select_format.select_by_visible_text(format)

                # create directory for each format
                path_format = os.path.join(path_device, format)
                print("path format: ", path_format)

                try:
                    os.mkdir(path_format)
                    print("Directory '%s' created successfully" % path_format)
                except OSError as error:
                    print("Directory '%s' already exists" % path_format)


                time.sleep(1)
                select_iso = Select( wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[3]/div/select"))))
                iso_number = len(select_iso.options)
                print("iso_number ",iso_number)
                
                for iso_itterator in range(itterator_iso, iso_number):
                    time.sleep(1)
                    select_iso = Select(wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[1]/div[6]/div/div[1]/div/div/div[2]/div[3]/div[2]/div[2]/div[1]/div[3]/div/select"))))
                    time.sleep(1)
                    iso_number = select_iso.options[iso_itterator].text
                    select_iso.select_by_visible_text(iso_number)


                    #get option text
                    print(iso_number)
                    select_iso.select_by_visible_text(iso_number)

                    # create directory for each iso value
                    path_iso = os.path.join(path_format, iso_number)
                    print("path iso: ", path_iso)
                    print("path_num", device_num)
                    try:
                        os.mkdir(path_iso)
                        print("Directory '%s' created successfully" % path_iso)
                    except OSError as error:
                        print("Directory '%s' already exists" % path_iso)
                        if os.listdir(path_iso) != []:
                            continue

                    #download image and put into folder (Device, Format, Iso, Format)
                    download_image(driver, format)

                    move_download(path_iso)
                    time.sleep(1)
                    itterator_iso += 1
                itterator_raw += 1
            itterator_stop += 1
    except:
        # if a 429 error is raised, wait for the specified time and try again
        print("Received a 429 error. Retrying after {} seconds...".format(wait_time))
        time.sleep(wait_time)
